In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 711MB/s]


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.


In [8]:
# model_experiment_nbeats.ipynb

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import mlflow
import mlflow.pytorch
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

class NBEATSBlock(nn.Module):
    def __init__(self, input_size, theta_size, basis_function, layers, layer_size):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, layer_size)] +
                                    [nn.Linear(layer_size, layer_size) for _ in range(layers-1)])
        self.basis_function = basis_function
        self.backcast_fc = nn.Linear(layer_size, theta_size)
        self.forecast_fc = nn.Linear(layer_size, theta_size)

    def forward(self, x):
        # Forward through fully connected layers
        for layer in self.layers:
            x = torch.relu(layer(x))

        # Generate theta parameters
        backcast_theta = self.backcast_fc(x)
        forecast_theta = self.forecast_fc(x)

        # Apply basis functions
        backcast, forecast = self.basis_function(backcast_theta, forecast_theta)

        return backcast, forecast

class GenericBasis(nn.Module):
    def __init__(self, backcast_size, forecast_size):
        super().__init__()
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Generic basis function - fixed size handling
        batch_size = backcast_theta.shape[0]
        backcast = backcast_theta[:, :self.backcast_size]
        forecast = forecast_theta[:, :self.forecast_size]

        # Ensure correct dimensions
        if backcast.shape[1] < self.backcast_size:
            padding = torch.zeros(batch_size, self.backcast_size - backcast.shape[1], device=backcast.device)
            backcast = torch.cat([backcast, padding], dim=1)

        if forecast.shape[1] < self.forecast_size:
            padding = torch.zeros(batch_size, self.forecast_size - forecast.shape[1], device=forecast.device)
            forecast = torch.cat([forecast, padding], dim=1)

        return backcast, forecast

class SeasonalityBasis(nn.Module):
    def __init__(self, harmonics, backcast_size, forecast_size):
        super().__init__()
        self.harmonics = harmonics
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Seasonality basis using Fourier series
        batch_size = backcast_theta.shape[0]
        device = backcast_theta.device

        # Create time indices on the correct device
        backcast_time = torch.linspace(0, 1, self.backcast_size, device=device).unsqueeze(0).repeat(batch_size, 1)
        forecast_time = torch.linspace(0, 1, self.forecast_size, device=device).unsqueeze(0).repeat(batch_size, 1)

        backcast = torch.zeros_like(backcast_time)
        forecast = torch.zeros_like(forecast_time)

        for i in range(self.harmonics):
            cos_coef = backcast_theta[:, i].unsqueeze(-1)
            sin_coef = backcast_theta[:, i + self.harmonics].unsqueeze(-1)

            # Apply Fourier basis
            backcast += (cos_coef * torch.cos(2 * np.pi * (i + 1) * backcast_time) +
                        sin_coef * torch.sin(2 * np.pi * (i + 1) * backcast_time))

            cos_coef_f = forecast_theta[:, i].unsqueeze(-1)
            sin_coef_f = forecast_theta[:, i + self.harmonics].unsqueeze(-1)

            forecast += (cos_coef_f * torch.cos(2 * np.pi * (i + 1) * forecast_time) +
                        sin_coef_f * torch.sin(2 * np.pi * (i + 1) * forecast_time))

        return backcast, forecast

class TrendBasis(nn.Module):
    def __init__(self, degree_of_polynomial, backcast_size, forecast_size):
        super().__init__()
        self.degree = degree_of_polynomial
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Polynomial trend basis
        batch_size = backcast_theta.shape[0]
        device = backcast_theta.device

        backcast_time = torch.linspace(-1, 1, self.backcast_size, device=device).unsqueeze(0).repeat(batch_size, 1)
        forecast_time = torch.linspace(-1, 1, self.forecast_size, device=device).unsqueeze(0).repeat(batch_size, 1)

        backcast = torch.zeros_like(backcast_time)
        forecast = torch.zeros_like(forecast_time)

        for i in range(self.degree + 1):
            backcast += backcast_theta[:, i].unsqueeze(-1) * (backcast_time ** i)
            forecast += forecast_theta[:, i].unsqueeze(-1) * (forecast_time ** i)

        return backcast, forecast


class NBEATS(nn.Module):
    def __init__(self, backcast_length, forecast_length,
                 stack_types=['generic', 'seasonality', 'trend'],
                 nb_blocks_per_stack=3, hidden_layer_units=128,
                 nb_harmonics=10, polynomial_degree=3):
        super().__init__()
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.hidden_layer_units = hidden_layer_units
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.stack_types = stack_types

        self.stacks = nn.ModuleList()

        for stack_type in stack_types:
            stack_blocks = nn.ModuleList()

            for _ in range(nb_blocks_per_stack):
                if stack_type == 'generic':
                    theta_size = backcast_length + forecast_length
                    basis_function = GenericBasis(backcast_length, forecast_length)
                elif stack_type == 'seasonality':
                    theta_size = 2 * nb_harmonics
                    basis_function = SeasonalityBasis(nb_harmonics, backcast_length, forecast_length)
                elif stack_type == 'trend':
                    theta_size = polynomial_degree + 1
                    basis_function = TrendBasis(polynomial_degree, backcast_length, forecast_length)

                block = NBEATSBlock(
                    input_size=backcast_length,
                    theta_size=theta_size,
                    basis_function=basis_function,
                    layers=4,
                    layer_size=hidden_layer_units
                )
                stack_blocks.append(block)

            self.stacks.append(stack_blocks)

    def forward(self, backcast):
        # Initialize forecast with correct device
        forecast = torch.zeros(backcast.size(0), self.forecast_length, device=backcast.device)

        for stack in self.stacks:
            for block in stack:
                b, f = block(backcast)
                backcast = backcast - b
                forecast = forecast + f

        return forecast

class WalmartDataset(Dataset):
    def __init__(self, data, backcast_length, forecast_length):
        self.data = data
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length

    def __len__(self):
        return len(self.data) - self.backcast_length - self.forecast_length + 1

    def __getitem__(self, idx):
        backcast = self.data[idx:idx + self.backcast_length]
        forecast = self.data[idx + self.backcast_length:idx + self.backcast_length + self.forecast_length]
        return torch.FloatTensor(backcast), torch.FloatTensor(forecast)

def train_nbeats(model, train_loader, val_loader, epochs=100, lr=0.001, device='cpu'):
    """
    Training loop for the N-BEATS model.

    Args:
        model (nn.Module): The N-BEATS model instance.
        train_loader (DataLoader): DataLoader for the training set.
        val_loader (DataLoader): DataLoader for the validation set.
        epochs (int): Number of training epochs.
        lr (float): Learning rate.
        device (str): Device to use for training.

    Returns:
        train_losses (list): Epoch-wise training losses.
        val_losses (list): Epoch-wise validation losses.
    """

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_losses = []
    val_losses = []

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0

        # Training loop
        for backcast, forecast in train_loader:
            # Move tensors to the correct device
            backcast, forecast = backcast.to(device), forecast.to(device)

            optimizer.zero_grad()
            outputs = model(backcast)
            loss = criterion(outputs, forecast)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

        avg_train_loss = running_train_loss / max(1, len(train_loader))
        train_losses.append(avg_train_loss)

        # Validation loop
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for backcast, forecast in val_loader:
                # Move tensors to the correct device
                backcast, forecast = backcast.to(device), forecast.to(device)

                outputs = model(backcast)
                loss = criterion(outputs, forecast)
                running_val_loss += loss.item()

        avg_val_loss = running_val_loss / max(1, len(val_loader))
        val_losses.append(avg_val_loss)

        # Periodic logging
        if epoch % 10 == 0 or epoch == 1 or epoch == epochs:
            print(f"[Epoch {epoch}/{epochs}] Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

    return train_losses, val_losses

def run_nbeats_experiment():
    """N-BEATS experiment runner"""

    # Check if CUDA is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load data - add error handling
    try:
        df = pd.read_csv('train.csv')
        df['Date'] = pd.to_datetime(df['Date'])
        print(f"Data loaded successfully. Shape: {df.shape}")
    except FileNotFoundError:
        print("Error: train.csv file not found. Please ensure the file exists in the current directory.")
        return None, None, None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None

    # MLflow experiment setup
    mlflow.set_experiment("Walmart_NBEATS_Forecasting")

    with mlflow.start_run():
        # Parameters
        BACKCAST_LENGTH = 52  # 52 weeks
        FORECAST_LENGTH = 4   # 4 weeks forecast
        BATCH_SIZE = 32
        EPOCHS = 200
        LEARNING_RATE = 0.001

        # Log parameters
        mlflow.log_param("backcast_length", BACKCAST_LENGTH)
        mlflow.log_param("forecast_length", FORECAST_LENGTH)
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("learning_rate", LEARNING_RATE)
        mlflow.log_param("device", str(device))

        # Prepare dataset
        weekly_sales = df.groupby('Date')['Weekly_Sales'].sum().values
        print(f"Weekly sales data points: {len(weekly_sales)}")

        # Check if we have enough data
        min_required = BACKCAST_LENGTH + FORECAST_LENGTH
        if len(weekly_sales) < min_required:
            print(f"Error: Not enough data points. Need at least {min_required}, got {len(weekly_sales)}")
            return None, None, None

        # Normalize data
        sales_mean = weekly_sales.mean()
        sales_std = weekly_sales.std()
        weekly_sales_normalized = (weekly_sales - sales_mean) / sales_std

        mlflow.log_param("sales_mean", sales_mean)
        mlflow.log_param("sales_std", sales_std)

        # Train/Val/Test split
        train_size = int(0.7 * len(weekly_sales_normalized))
        val_size = int(0.15 * len(weekly_sales_normalized))

        train_data = weekly_sales_normalized[:train_size]
        val_data = weekly_sales_normalized[train_size:train_size + val_size]
        test_data = weekly_sales_normalized[train_size + val_size:]

        print(f"Train size: {len(train_data)}, Val size: {len(val_data)}, Test size: {len(test_data)}")

        # DataLoaders
        train_dataset = WalmartDataset(train_data, BACKCAST_LENGTH, FORECAST_LENGTH)
        val_dataset = WalmartDataset(val_data, BACKCAST_LENGTH, FORECAST_LENGTH)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Model initialization
        model = NBEATS(
            backcast_length=BACKCAST_LENGTH,
            forecast_length=FORECAST_LENGTH,
            stack_types=['trend', 'seasonality', 'generic'],
            nb_blocks_per_stack=3,
            hidden_layer_units=128
        ).to(device)

        print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")

        # Training
        train_losses, val_losses = train_nbeats(model, train_loader, val_loader, EPOCHS, LEARNING_RATE, device)

        # Log metrics
        mlflow.log_metric("final_train_loss", train_losses[-1])
        mlflow.log_metric("final_val_loss", val_losses[-1])

        # Save model
        mlflow.pytorch.log_model(model, "nbeats_model")

        # Testing
        if len(test_data) >= BACKCAST_LENGTH + FORECAST_LENGTH:
            test_dataset = WalmartDataset(test_data, BACKCAST_LENGTH, FORECAST_LENGTH)
            test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

            model.eval()
            predictions = []
            actuals = []

            with torch.no_grad():
                for backcast, forecast in test_loader:
                    backcast, forecast = backcast.to(device), forecast.to(device)
                    pred = model(backcast)

                    # Denormalize predictions and actuals
                    pred_denorm = pred.cpu().numpy() * sales_std + sales_mean
                    actual_denorm = forecast.cpu().numpy() * sales_std + sales_mean

                    predictions.extend(pred_denorm.flatten())
                    actuals.extend(actual_denorm.flatten())

            # Test metrics
            mae = mean_absolute_error(actuals, predictions)
            mse = mean_squared_error(actuals, predictions)
            rmse = np.sqrt(mse)

            mlflow.log_metric("test_mae", mae)
            mlflow.log_metric("test_mse", mse)
            mlflow.log_metric("test_rmse", rmse)

            print(f"Test MAE: {mae:.2f}")
            print(f"Test RMSE: {rmse:.2f}")

            return model, predictions, actuals
        else:
            print("Not enough test data for evaluation")
            return model, [], []

if __name__ == "__main__":
    model, predictions, actuals = run_nbeats_experiment()

Using device: cuda
Data loaded successfully. Shape: (421570, 5)
Weekly sales data points: 143
Train size: 100, Val size: 21, Test size: 22
Model initialized with 568800 parameters


ValueError: __len__() should return >= 0